In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
df = pd.read_csv(
    '/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',
    encoding='ISO-8859-1',
    header=None
)
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df.dtypes

target     int64
ids        int64
date      object
flag      object
user      object
text      object
dtype: object

In [4]:
for col in df.columns :
    print("#####################")
    print( df[col].value_counts())

#####################
target
0    800000
4    800000
Name: count, dtype: int64
#####################
ids
1972193428    2
1989776729    2
1989776908    2
1564543229    2
1690832844    2
             ..
2197310477    1
2197310452    1
2197310381    1
2197310235    1
2197311763    1
Name: count, Length: 1598315, dtype: int64
#####################
date
Mon Jun 15 12:53:14 PDT 2009    20
Fri May 22 05:10:17 PDT 2009    17
Mon Jun 15 13:39:50 PDT 2009    17
Fri May 29 13:40:04 PDT 2009    17
Fri Jun 05 14:13:07 PDT 2009    16
                                ..
Tue Jun 16 08:34:52 PDT 2009     1
Mon Jun 15 14:18:18 PDT 2009     1
Mon Jun 15 14:18:19 PDT 2009     1
Mon Jun 15 14:18:21 PDT 2009     1
Mon Jun 15 14:18:23 PDT 2009     1
Name: count, Length: 774363, dtype: int64
#####################
flag
NO_QUERY    1600000
Name: count, dtype: int64
#####################
user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
       

In [5]:
for col in df.columns :
    print("#####################")
    print(col, ":", df[col].nunique())

#####################
target : 2
#####################
ids : 1598315
#####################
date : 774363
#####################
flag : 1
#####################
user : 659775
#####################
text : 1581466


In [6]:
df[['dow', 'month', 'day', 'time', 'tz', 'year']] = df['date'].str.split(expand=True)
df['day'] = df['day'].astype(int)
df['year'] = df['year'].astype(int)

df.drop(columns=["flag", "date"], inplace=True)

In [7]:
df.head()

,target,ids,user,text,dow,month,day,time,tz,year
0,0,1467810369,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Mon,Apr,6,22:19:45,PDT,2009
1,0,1467810672,scotthamilton,is upset that he can't update his Facebook by ...,Mon,Apr,6,22:19:49,PDT,2009
2,0,1467810917,mattycus,@Kenichan I dived many times for the ball. Man...,Mon,Apr,6,22:19:53,PDT,2009
3,0,1467811184,ElleCTF,my whole body feels itchy and like its on fire,Mon,Apr,6,22:19:57,PDT,2009
4,0,1467811193,Karoli,"@nationwideclass no, it's not behaving at all....",Mon,Apr,6,22:19:57,PDT,2009


In [8]:
df.dtypes

target     int64
ids        int64
user      object
text      object
dow       object
month     object
day        int64
time      object
tz        object
year       int64
dtype: object

In [9]:

df['user'] = df['user'].astype('string')
df['text'] = df['text'].astype('string')
df['dow'] = df['dow'].astype('category')
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
df['tz'] = df['tz'].astype('category')


month_map = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}
df['month'] = df['month'].map(month_map)

target_map = {0: 0, 2: 1, 4: 2}

df['target'] = df['target'].map(target_map)




In [10]:
df.dtypes

target             int64
ids                int64
user      string[python]
text      string[python]
dow             category
month              int64
day                int64
time              object
tz              category
year               int64
dtype: object

In [11]:
type(df['time'][0])


datetime.time

In [12]:
for col in df.columns :
    print("#####################")
    print( df[col].value_counts())

#####################
target
0    800000
2    800000
Name: count, dtype: int64
#####################
ids
1972193428    2
1989776729    2
1989776908    2
1564543229    2
1690832844    2
             ..
2197310477    1
2197310452    1
2197310381    1
2197310235    1
2197311763    1
Name: count, Length: 1598315, dtype: int64
#####################
user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
                  ... 
ainachy              1
Leleah               1
CanterburyVilla      1
Bitch_girl           1
dwyarianne           1
Name: count, Length: 659775, dtype: Int64
#####################
text
isPlayer Has Died! Sorry                                                                                                         210
good morning                                                                                                                     118
headache                                                     

In [13]:
for col in df.columns :
    print("#####################")
    print(col, ":", df[col].nunique())

#####################
target : 2
#####################
ids : 1598315
#####################
user : 659775
#####################
text : 1581466
#####################
dow : 7
#####################
month : 3
#####################
day : 29
#####################
time : 86386
#####################
tz : 1
#####################
year : 1


# Model

In [14]:
!pip install transformers datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found 

In [15]:

df_sample = df.sample(frac=0.1, random_state=42).reset_index(drop=True)
df_sample = df_sample[['text', 'target']]

In [16]:
# Replace URLs and mentions (optional but recommended)
import re

def preprocess(text):
    text = re.sub(r"http\S+", "<URL>", text)
    text = re.sub(r"@\S+", "<USER>", text)
    return text

df_sample['text'] = df['text'].apply(preprocess)


In [17]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_sample['text'].tolist(), df_sample['target'].tolist(), test_size=0.2, random_state=42
)


In [18]:
from transformers import AutoTokenizer

MODEL_NAME = "cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [19]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)


2025-11-04 22:34:43.039641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762295683.271991      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762295683.340864      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [21]:
from torch.utils.data import DataLoader
from tqdm import tqdm

model.eval()  # Set model to evaluation mode

test_loader = DataLoader(test_dataset, batch_size=32)
preds = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_preds = torch.argmax(logits, dim=1)
        preds.extend(batch_preds.tolist())


  0%|          | 0/1000 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

100%|██████████| 1000/1000 [43:44<00:00,  2.62s/it]


In [22]:
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(test_labels, preds)
print("Accuracy:", accuracy)

print(classification_report(test_labels, preds, target_names=["Negative", "Neutral", "Positive"]))


Accuracy: 0.3545
              precision    recall  f1-score   support

    Negative       0.50      0.56      0.52     15878
     Neutral       0.00      0.00      0.00         0
    Positive       0.51      0.16      0.24     16122

    accuracy                           0.35     32000
   macro avg       0.34      0.24      0.25     32000
weighted avg       0.50      0.35      0.38     32000



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
